In [28]:
import pandas as pd
import numpy as np
import pickle as p

In [29]:
data_all = pd.read_csv('data_all.csv')

# Test with Subset of Data
np.random.seed(511)
item_subset = list(np.random.choice(data_all['Item_ID'].unique(), size=10, replace=False))
data_subset = data_all.loc[data_all['Item_ID'].isin(item_subset)].copy()

# # Test with All Data
# data_subset = data_all.copy()

In [30]:
item_subset

[5662, 1230, 5820, 1002, 3743, 2736, 1203, 3772, 16018, 3768]

### `/train`

In [31]:
import requests
import json

url = 'https://fillet.azurewebsites.net/train/'

data = data_subset.to_json()

payload = {'data': data,
           'cv_acc': False,
           'project_id': 35
           }

headers = {'content-type': 'application/json',
           'Accept-Charset': 'UTF-8'
          }

payload = json.dumps(payload)
r = requests.post(url, data=payload, headers=headers, timeout=1200)

In [32]:
r.json()['result']

'Success'

In [ ]:
# CV Accuracies Returned
cv_accuracies = pd.DataFrame.from_dict((r.json()['cv_acc']))
cv_accuracies

### `/optimize`

In [ ]:
url = 'http://localhost:5000/optimize/'

payload = {'project_id': 1,
           'constraints': [],
           'population': 100,
           'max_epoch': 200
           }

headers = {'content-type': 'application/json',
           'Accept-Charset': 'UTF-8'
          }

payload = json.dumps(payload)
r = requests.post(url, data=payload, headers=headers)

In [ ]:
best_prices = r.json()['result']
best_prices

In [ ]:
import requests
import json

url = 'http://localhost:5000/predict/'
payload = {'prices': best_prices,
           'project_id': 1
           }
headers = {'content-type': 'application/json',
           'Accept-Charset': 'UTF-8'
          }
payload = json.dumps(payload)
r = requests.post(url, data=payload, headers=headers)

# What If Prediction
qty = r.json()['qty_estimates']

In [ ]:
qty

In [ ]:
# Calculate Revenue
revenue = 0
for item in best_prices.keys():
    revenue += (best_prices[item]*qty['Qty_'+item])
    
print(revenue)

### `/predict/`

In [34]:
# Sample Actual Price 
from helper_functions import *

sales_data = data_subset

sales_data = optimize_memory(sales_data)

sales_data_wide = sales_data.set_index(
    ['Wk', 'Tier', 'Store', 'Item_ID']).unstack(level=-1).reset_index().copy()
sales_data_wide.columns = [
    ''.join(str(i) for i in col).strip()
    for col in sales_data_wide.columns.values
]
sales_data_wide = sales_data_wide.sort_values(
    ['Tier', 'Store', 'Wk'], ascending=True).reset_index(drop=True)

sales_data_wide_clean = sales_data_wide.dropna(axis=0).copy()
dataset = sales_data_wide_clean
price_columns = [
    col for col in sales_data_wide_clean.columns if col.startswith('Price')
]

a = dataset.sample(1)
a_dict = a[price_columns].iloc[0].to_dict()
a_input = {}
for key in a_dict.keys():
    a_input[key.split('_')[1]]=round(a_dict[key],2)

print('Input Prices Quantities:')
a_input

Input Prices Quantities:


{'1002': 2.9,
 '1203': 3.15,
 '1230': 4.0,
 '2736': 10.5,
 '3743': 11.85,
 '3768': 11.7,
 '3772': 10.25,
 '5662': 0.7,
 '5820': 2.5,
 '16018': 13.45}

In [ ]:
# Predict Sales Qty and Compare to Actual

import requests
import json

url = 'https://fillet.azurewebsites.net/predict/'

payload = {'prices': a_input, 'project_id': 35}

headers = {'content-type': 'application/json', 'Accept-Charset': 'UTF-8'}

payload = json.dumps(payload)

r = requests.post(url, data=payload, headers=headers)
print(r.status_code)
qty = r.json()['qty_estimates']
qty_outp = {}
for key in qty.keys():
    qty_outp[key] = round(qty[key], 0)
actual = a[[
    col for col in sales_data_wide_clean.columns if col.startswith('Qty_')
]].iloc[0].to_dict()
pd.DataFrame([qty_outp, actual], index=['Estimated', 'Actual']).transpose()